In [8]:
import random as rd
from math import *


def max_pui_deux(n):
    if n == 0:
        return 0
    i = 1
    while n % i != 0:
        i *= 2
    return i


'''
    Compute x^n
    Parameters :
    - A number x
    - A power n
'''


def quickExponent(x, n):
    # Base x cases
    if x == 0:
        return 0
    if x == 1:
        return 1
    # Base n cases
    if n == 0:
        return 1
    if n == 1:
        return x
    # Heredity
    if (n % 2 == 0):
        return (quickExponent(x, n//2) ** 2)
    else:
        return (quickExponent(x, n//2) ** 2) * x


'''
    Compute x^n %m
    Parameters :
        - A number x
        - A power n
        - A modulus m
'''


def quickModularExponent(x, n, m):
    # Base x cases
    if x == 0:
        return 0
    if x == 1:
        return 1
    # Base n cases
    if n == 0:
        return 1
    if n == 1:
        return x % m
    # modulo 1 case
    if m == 1:
        return 1
    # Heredity
    if (n % 2 == 0):
        return (quickModularExponent(x, n//2, m) ** 2) % m
    else:
        return ((quickModularExponent(x, n//2, m) ** 2) * x) % m


def miller_rabin(n, k, m):
    if n == 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    # n = 1 + m*2^k, where m is odd,
    # i.e. 2^k is the highest power of 2 dividing n-1, m being the other factor
    a = rd.randint(2, n-1)
    if quickModularExponent(a, m, n) == 1 or quickModularExponent(a, m, n) == n-1:
        return True
    for i in range(1, k):
        if quickModularExponent(a, 2*i*m, n) == n-1:
            return True
    return False


def isProbablyPrime(n, likelihood=99):
    k = max_pui_deux(n-1)
    m = int((n-1)/(quickExponent(2, k)))
    # Base error rate for a single miller-rabin test is about 1/4.
    # So false-positive probability after j tests is 1 - 4^(-j)
    # To reach the given likelihood l, we need 1 - 4^(-j) >= l/100
    # 4^(-j) <= (100-l)/100
    # -j <= log4(100 - l) - log4(100)
    # j >= log4(100) - log4(100 - l)
    # log4(100) is about 3.32, let's round it to 4
    for i in range(floor(4 - log(100 - likelihood, 4))):
        if not miller_rabin(n, k, m):
            return False
    return True


''' 
    TESTS
'''


def main():
    print(quickModularExponent(78, 4567, 154))

    print(miller_rabin(33, 4, 2))  # 33 is 2*2^4 + 1
    print(miller_rabin(31, 1, 15))  # 31 is 15*2^1 + 1
    print(miller_rabin(17, 3, 2))  # 17 is 2*2^3 + 1
    print(miller_rabin(3, 1, 1))  # 3 is 1*2^1 + 1 ???

    print(isProbablyPrime(3))
    print(isProbablyPrime(17))
    print(isProbablyPrime(31))


if __name__ == "__main__":
    main()


78
False
True
True
True
True
True
True


In [1]:
from MR import isProbablyPrime, quickModularExponent
from math import gcd
import random

'''
    Global variables for key size
'''
byte_size = 8


'''
    Generates a prime number. Uses Miller-Rabin as primality test.
'''


def generate_prime(size=byte_size):
    r = int.from_bytes(random.randbytes(size), byteorder="little")
    while(not isProbablyPrime(r)):
        r = int.from_bytes(random.randbytes(size), byteorder="little")
    return r


'''
    Finds a number coprime with phi
    Parameters :
        - The value phi = (p-1) * (q-1)

    Returns :
        A number e such as 1 < e < phi(n) and gcd(e,phi(n)) = 1

'''


def find_coprime(phi):
    for i in range(2, phi):
        if (gcd(i, phi) == 1):
            return i
    return -1


'''
    Find the phi modular inverse of e using Euclide's algorithm
    Parameters :
        - The number e
        - The modulo phi
    Returns
        A number d such as ed = 1 mod phi
'''


def find_inverse(e, phi):
    m = phi
    x = 1
    y = 0

    if(phi == 1):
        return 0
    while (e > 1):
        q = e//phi
        t = phi

        phi, e, t = e % phi, t, y
        y, x = x-q*t, t

    if (x < 0):
        x += m
    return x


'''
    Generates public and private keys for RSA

    Returns :
        A list [(n,e),(d,n)] containing the public and the private keys
'''


def generate_RSA(DES=False):
    if (DES):
        p = generate_prime(4)
        q = generate_prime(3)
        while (q == p):
            q = generate_prime()
    else:
        p, q = generate_prime(), generate_prime()
        while (q == p and 2**56 > p*q and p*q > 2**57):
            q = generate_prime()

    n = p*q
    phi = (p-1) * (q-1)
    e = find_coprime(phi)

    d = find_inverse(e, phi)
    return [[n, e], [d, n]]


'''
    Encrypts a message using RSA
    Parameters :
        - msg : the message to encrypt
        - key : the public key (n,e)

    Returns :
        The encrypted message
'''


def encrypt_RSA(msg, key):

    cyphered = quickModularExponent(
        int.from_bytes(bytes(msg, 'UTF-8'), byteorder="little"), key[1], key[0]
    )
    return cyphered


'''
    Decrypt a message using RSA
    Parameters :
        - msg : the message to decrypt in int format
        - key : the private key (d,n)

    Returns :
        The raw message
'''


def decrypt_RSA(msg, key):
    mod = quickModularExponent(msg, key[0], key[1])
    print(mod)
    return mod.to_bytes(64, byteorder="little").decode("utf-8")


'''
    TESTS
'''


def main():
    print("Generated keys : ")
    k = generate_RSA()
    print(k)

    print("Encrypt 'HELLO' :")
    m = encrypt_RSA("HELLO", k[0])
    print("Encrypted message in int format :%d" % m)

    print("Decrypt 'HELLO' : ")
    print(decrypt_RSA(m, k[1]))


if __name__ == "__main__":
    main()


Generated keys : 
[[1824286091600716451666333498017164097, 5], [1459428873280573156705132919384231117, 1824286091600716451666333498017164097]]
Encrypt 'HELLO' :
Encrypted message in int format :1169280118516609042868188977608866994
Decrypt 'HELLO' : 
340582483272
HELLO                                                           


In [1]:
from random import randbytes
from math import gcd
from RSA import *
from MR import quickModularExponent
from Crypto.Cipher import AES
'''
    Bit stuffs a text in order to make its length a multiple of 16
    Parameter :
        - The raw text
    Returns :
        The bit stuffed raw text
        (For unknown reasons, modifying the text without returning it results in an unchanged text)
'''


def stuffing(rawtext):
    if len(rawtext) == 16:
        rawtext += 16*chr(16)
    else:
        r = len(rawtext) % 16
        r = 16-r
        rawtext += (r)*chr(r)
    return rawtext


'''
    Hybrid text encryption function using AES and RSA
    Parameters :
        - The raw text
        - The public key generated by RSA
    Returns :
        The cyphered text
'''


def encrypt_hybrid_AES(text, rsa_pubkey):

    # 16 hardcoded because it's AES
    k = randbytes(16)
    cyphered_key = encrypt_RSA(str(k), rsa_pubkey)

    aes = AES.new(k)
    text = stuffing(text)
    cyphered_message = int.from_bytes(
        aes.encrypt(text.encode()), byteorder="little")

    return cyphered_key, cyphered_message


'''
    Pollard's factorisation function
    WARNING : This function can make the program run in an endless loop
    Parameters :
        - A number n
    Retuns :
        p,q two prime numbers such as n = pq
'''


def pollard(n):
    def f(x): return x*x+1
    x, y, d = 2, 2, 1
    while d == 1:
        x = f(x) % n
        y = f(f(x)) % n
        d = gcd(x-y, n)
    return d, n//d


'''
    Supposing that :
        - we use DES functions with a key on 56 bits
        - 2**56<= n=pq <=2**57
        - we have the public key (n,e)
    Crack the key

    Parameters:
        - The encrypted key
        - The public key generated by RSA

    Returns :
        The decrypted key
'''


def decrypt_DES(des_key, pub_key):
    p, q = pollard(pub_key[0])
    phi = (p-1)(q-1)

    d = find_inverse(pub_key[1], phi)

    return decrypt_RSA(des_key, (pub_key[0], d))


'''
    TESTS
'''


def main():

    # Testing hybrid encryption
    print("Hybrid Encrypting 'HELLO' : ")
    k = generate_RSA()
    key, message = encrypt_hybrid_AES("HELLO", k[0])
    print("Encrypted AES key = ", key)
    print("Encrypted message in int format = ", message)

    # Testing DES
    des_key = randbytes(7)
    encrypted_key = encrypt_RSA(str(des_key), k[0])
    print("Encrypted DES key = ", encrypted_key)
    print("Decrypted DES key = ", decrypt_DES(encrypted_key, k[0]))

    return


if __name__ == "__main__":
    main()


Hybrid Encrypting 'HELLO' : 
Encrypted AES key =  14834344330559000934087898885886
Encrypted message in int format =  281748193342802669730062645493351171944
Encrypted DES key =  18003645536654906769234516670824
